In [ ]:
import os
import numpy as np

## 5.19 Rotate a 2D Array 

Image rotation is a fundamental operation in computer graphics. Figure 5.4 illustrates the rotation operation on a 2D array representing a bit-map of an image. Specifically, the image is rotated by 90 degrees clockwise.

Write a function that takes as input an n x n 2D array, and rotates the array by 90 degrees clockwise.
Hint: Focus on the boundary elements.

In [ ]:
def rotate_90_clock_np(arr):
    arr = np.array(arr)
    arr_rot = np.zeros(arr.shape)
    for old_row_ind, new_col_ind in enumerate(range(len(arr))[::-1]):
        arr_rot[:, new_col_ind] = arr[old_row_ind, :]
    return arr_rot


def rotate_coords_90_clock(i, j, n):
    i_rot = j
    j_rot = n - 1 - i
    return i_rot, j_rot


def rotate_corners(arr, i_start, j_start):
    n = len(arr)
    i_orig, j_orig = i_start, j_start
    reassign_val = arr[i_orig][j_orig]
    for _ in range(4):
        i_rot, j_rot = rotate_coords_90_clock(i_orig, j_orig, n)
        temp = arr[i_rot][j_rot]
        arr[i_rot][j_rot] = reassign_val
        reassign_val = temp
        i_orig, j_orig = i_rot, j_rot

        
def rotate_square(arr, ind_start):
    i_start = ind_start
    for j_start in range(ind_start, len(arr) - ind_start - 1):
        rotate_corners(arr, i_start, j_start)

        
def rotate_90_clock_inplace(arr):
    num_square = len(arr) / 2
    for ind_start in range(num_square):
        rotate_square(arr, ind_start)

        
def rotate_90_clock(arr):
    n = len(arr)
    arr_rot = [[None] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            i_rot, j_rot = rotate_coords_90_clock(i, j, n)
            arr_rot[i_rot][j_rot] = arr[i][j]
    return arr_rot


def rotate(arr):
    for i in xrange(len(arr)/2):
#         for j in xrange(len(arr)/2 + len(arr)%2):
        for j in xrange(i, len(arr) - i - 1):
            rotate_corners(arr,i,j)
    return arr

In [ ]:
# arr = [[i] * n for i in range(5)]
# arr = [[1, 2], [3, 4]]
# arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
arr = [[1, 2, 3, 4, 140], [5, 6, 7, 8, 130], [9, 10, 11, 12, 120], [13, 14, 15, 16, 110], [17, 18, 19, 20, 100]]

# arr_rot = rotate_90_clock(arr)

# print(np.array(arr))
# print('')
# print(np.array(arr_rot))

print(np.array(arr))
# rotate_90_clock_inplace(arr)
rotate(arr)
print('')
print(np.array(arr))

## 5.13 Sample Online Data 

This problem is motivated by the disgn of a packet sniffer that provides a uniform sample of packets for a network session.

Design a program that takes as input a size k, and reads packets, coninuously maintaining a uniform random subset of size k of the read packets.

Hint: Suppose you have a procedure which selects k packets from the first n >= k packets as specified. How would you deal with the (n+1)st packet?

In [ ]:
import random

class PacketSampler:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = float(self.k) / (self.n + 1)
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


class PacketSamplerWrong:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = random.random()
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


def get_packet_subset(n, packet_sampler):
    for i in range(n):
        packet_sampler.receive_packet(i)
    return packet_sampler.subset


def get_simulated_distribution(n, k, sampler_name, num_trials):
    if sampler_name == 'right':
        Sampler = PacketSampler
    elif sampler_name == 'wrong':
        Sampler = PacketSamplerWrong
    else:
        raise ValueError('Unrecognized sampler name.')
    
    packet_counter = {p: 0 for p in range(n)}
    for trial_ind in range(num_trials):
        subset = get_packet_subset(n, Sampler(k))
        for p in subset:
            packet_counter[p] += 1
    
    return packet_counter

In [ ]:
num_trials = 500000
n = 10
k = 3
sampler_name = 'wrong'

packet_counts = get_simulated_distribution(n=n, k=k, sampler_name=sampler_name, num_trials=num_trials)
num_exp = num_trials / n * k

for count in packet_counts.itervalues():
    pct_dev = 100.0 * abs(num_exp - count) / num_exp
    print(pct_dev)

## 5.18 Compute the Spiral Ordering of a 2D array 

In [ ]:
def change_direction(increment):
    increment_new = increment[::-1]
    if abs(increment[0]) == 1:
        increment_new[1] *= -1
    return increment_new

def spiralize(arr):
    i_bounds = [0, len(arr) - 1]
    j_bounds = [0, len(arr[0]) - 1]
    increment = [0, 1]
    i = 0
    j = 0
    arr_spiral = []

    while i_bounds[1] >= i_bounds[0]:

        arr_spiral.append(arr[i][j])

        i_prop = i + increment[0]
        j_prop = j + increment[1]

        if j_prop > j_bounds[1]:
            i_bounds[0] += 1
        elif j_prop < j_bounds[0]:
            i_bounds[1] -= 1
        elif i_prop > i_bounds[1]:
            j_bounds[1] -= 1
        elif i_prop < i_bounds[0]:
            j_bounds[0] += 1
        else:
            i = i_prop
            j = j_prop
            continue

        increment = change_direction(increment)
        i += increment[0]
        j += increment[1]
        
    return arr_spiral

In [ ]:
arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

print(np.array(arr))
print('')
print(spiralize(arr))

## 6.13: Find the first occurrence of a substring 

A good string search algorithm is fundamental to the performance of many applications. Several clever algorithms have been proposed for string search, each with its own trade-offs. As a result, there is no single perfect answer. If someone asks you this question in an interview, the best way to approach this problem would be to work through one good algorithm in detail and discuss at a high level other algorithms.

Given two strings s (the "search string") and t (the "text"), find the first occurrence of s in t.

Hint: Form a signature from a string

In [ ]:
from collections import deque


def strings_are_same(str1, str2):
    for c1, c2 in zip(str1, str2):
        if c1 != c2:
            return False
    return True


class RollingHasher:
    """Following https://en.wikipedia.org/wiki/Rabin%E2%80%93Karp_algorithm"""
    
    def __init__(self, substring_len, base=256, prime_modulus=101):
        self.substring_len = substring_len
        self.base = base
        self.prime_modulus = prime_modulus
        self._char_q = deque()
        self._hash = 0
        
    def _add_to_hash(self, c):
        # O(1) time
        self._hash = (self._hash * self.base + ord(c)) % self.prime_modulus
        self._char_q.append(c)
        
    def _remove_first_in_from_hash(self):
        # O(substring_len) time
        left_base_offset = 1
        for i in range(self.substring_len - 1):
            left_base_offset *= self.base 
            left_base_offset %= self.prime_modulus
        remove_val = ord(self._char_q.popleft())
        self._hash = self._hash + self.prime_modulus - remove_val * left_base_offset  # adding extra mod val ensures result isn't negative
        
    def get_hash(self, next_letter):
        # O(substring_len) time
        if len(self._char_q) == self.substring_len:
            hash_sub = self._remove_first_in_from_hash()
        self._add_to_hash(next_letter)
        return self._hash if len(self._char_q) == self.substring_len else None


def find_substring_start_ind_simple(main_string, substring):
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if strings_are_same(substring, main_string[start_ind:start_ind + substring_len]):
            return start_ind
    return None


def find_substring_start_ind_set(main_string, substring):
    substring_hash = set([substring])
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if main_string[start_ind:start_ind + substring_len] in substring_hash:
            return start_ind
    return None

        
def find_substring_start_ind_rabin_karp(main_string, substring):
    # O(substring_len * string_len)
    sub_hasher = RollingHasher(len(substring))
    substring_hash = [sub_hasher.get_hash(c) for c in substring][-1]
    main_hasher = RollingHasher(len(substring))
    for i, c in enumerate(main_string):
        end_ind = i+1
        start_ind = end_ind - len(substring)
        if substring_hash == main_hasher.get_hash(c) and strings_are_same(substring, main_string[start_ind:end_ind]):
            return start_ind
    return None

In [ ]:
main_string = 'gshgiuregbisnreipngseproingseonvdk;nfvdklldksfl'
substring = 'gbisn'

print(find_substring_start_ind_simple(main_string, substring))
print(find_substring_start_ind_set(main_string, substring))
print(find_substring_start_ind_rabin_karp(main_string, substring))

## 6.6: Reverse all the words in a sentence

Given a string containing a set of words separated by whitespace, we would like to transform it to a string in which the words appear in reverse order. For example, "Alice likes Bob" transforms to "Bob likes Alice". We do not need to keep the original string.

Implement a function for reversing the words in a string s.

Hint: It's difficult to solve this with one pass.

In [ ]:
def reverse_words_simple(sentence):
    words = []
    word = ''
    for c in sentence:
        if c == ' ':
            words.append(word)
            words.append(' ')
            word = ''
        else:
            word += c
    words.append(word)
        
    sentence_rev = ''
    while len(words) > 0:
        sentence_rev += words.pop()
        
    return sentence_rev

def reverse_words_prealloc(sentence):
    end_ind = len(sentence)
    word = ''
    sentence_rev = [' '] * len(sentence)
    for c in sentence:
        if c == ' ':
            start_ind = end_ind-len(word)
            sentence_rev[start_ind:end_ind] = word
            end_ind = start_ind - 1
            word = ''
        else:
            word += c
    sentence_rev[end_ind-len(word):end_ind] = word
        
    return ''.join(sentence_rev)

In [ ]:
sentence = 'Bob likes Alice'
print(reverse_words_simple(sentence))
print(reverse_words_prealloc(sentence))

## 6.7: Compute all mnemonics for a phone number 

Each digit, apart from 0 and 1, in a phone keypad corresponds to one of hte three or four letters of the alphabet, as shown in figure 6.1. Since words are easier to remember than number, it is natural to ask if a 7 or 10-digit phone number can be represented by a word. For examples, "2276696" corresponds to "ACRONYM" as well as "ABPOMZN",

Write a program which takes as input a phone number, specified as a string of digits, and returns all possible character sequences that correspond to the phone number. The cell phone keypad is specified by a mapping that takes a digit and returns the corresponding set of characters. The character sequences do not have to be legal words or phrases.

Hint: Use recursion.

In [ ]:
DIGIT_LETTERS = {
    '2': ['A', 'B', 'C'],
    '3': ['D', 'E', 'F'],
    '4': ['G', 'H', 'I'],
    '5': ['J', 'K', 'L'],
    '6': ['M', 'N', 'O'],
    '7': ['P', 'Q', 'R', 'S'],
    '8': ['T', 'U', 'V'],
    '9': ['W', 'X', 'Y', 'Z']
}


def get_mnemonics(digits):
    if len(digits) == 1:
        return DIGIT_LETTERS[digits]
    
    prev_sequences = get_mnemonics(digits[:-1])
    digit = digits[-1]
    letters = DIGIT_LETTERS[digit]

    new_sequences = []
    for seq in prev_sequences:
        for letter in letters:
            new_sequences.append(seq + letter)
        
    return new_sequences

In [ ]:
digits = '23'
print(get_mnemonics(digits))

## 6.4: Replace and Remove 

Consider the following two rules that are to be applied to an array of characters.
* Replace each 'a' by two 'd's
* Delete each entry containing a 'b'
For example, applying these rules to the array [a,c,d,b,c,a] results in the array [d,d,c,d,c,d,d]

Write a program which takes as input an array of characters, and removes each 'b' and replaces each 'a' by two 'd's. Specifically, along with the array, you are provided an integer-valued size. Size denotes the number of entries of hte array that the operation is to be applied to. You do not have to worry about preserving subsequent entries. For example, if the array is [a,b,a,c,_] and the size is 4, then you can return [d,d,d,d,c]. You can assume there is enough space int he array to hold the final result.

Hint: Consider performing multiple passes on s.

In [ ]:
from collections import deque


def replace_and_remove(arr, size):
    q = deque()

    for i, c in enumerate(arr[:size]):
        if c == 'a':
            q.append('d')
            q.append('d')
        elif c != 'b':
            q.append(c)
        arr[i] = q.popleft()
    
    while len(q) > 0:
        i += 1
        arr[i] = q.popleft()
        
    return arr


def replace_and_remove_inplace(arr, size):
    # Get rid of b's first to make sure there's enough room for the extra d's
    replace_ind = 0
    num_a = 0
    for i, c in enumerate(arr[:size]):
        arr[i] = '_'
        if c != 'b':
            arr[replace_ind] = c
            replace_ind += 1
            if c == 'a':
                num_a += 1
        else:
            size -= 1
    
    # Add in d's
    cur_ind = size + num_a - 1
    for c in arr[:size][::-1]:
        if arr[cur_ind] == 'a':
            arr[cur_ind] = 'd'
            arr[cur_ind - 1] = 'd'
            cur_ind -= 2
        else:
            arr[cur_ind] = c
            cur_ind -= 1

    return arr

In [ ]:
arr = ['a','b','a','c','_']
# arr = ['a','a','a','b','b','b']
size = 4
# print(replace_and_remove(arr, size))
print(replace_and_remove_inplace(arr, size))

## 7.1 Merge Two Sorted Lists 

Consider two singly linked lists in which each node holds a number. Assume the lists are sorted, i.e., number isn the list appear in ascending order within each list. The merge of the two lists is a list consisting of the nodes of the two lists in which numbers appear in ascending order. Merge is illustrated in Figure 7.3.

Write a program that takes two lists, assumed to be sorted, and returns their merge. The only field your program can change in a node is its next field.

Hint: Two sorted arrays can be merged using two indices. For lists, take care when one iterator reaches the end.

In [ ]:
def merge_lists(ll1, ll2):
    if ll2.value < ll1.value:
        ll1, ll2 = ll2, ll1
    ll1_cur = ll1
    ll2_cur = ll2
    
    while ll2_cur is not None:
        # Advance ll1 to just before ll2 becomes smaller
        while ll1_cur.next is not None and ll1_cur.next.value < ll2_cur.value:
            ll1_cur = ll1_cur.next
        
        # Add the rest of ll2 if at the end of ll1
        if ll1_cur.next is None and ll2_cur is not None:
            ll1_cur.next = ll2_cur
            break
        
        # Pop from ll2 and insert into ll1
        ll2_next = ll2_cur.next
        ll2_cur.next = ll1_cur.next
        ll1_cur.next = ll2_cur
        ll2_cur = ll2_next
            
    return ll1

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili2 = LinkedList([1, 2, 3, 8, 29, 1000])
lili1 = LinkedList([4, 18, 19, 72, 80, 99])
# lili2 = LinkedList([1, 2, 4, 7])
# lili1 = LinkedList([1, 4, 4, 5])

print(merge_lists(lili1, lili2).as_list())

## 7.3 Test for Cyclicity 

Although a linked list is supposed to be a sequence of nodes ending in null, it is possible to create a cycle in a linked list by making the next field of an element reference to one of the earlier nodes. 

Write a program that takes the head of a singly linked list and returns null if there does not exist a cycle, and the node at the start of the cycle, if a cycle is present. (You do not know the length of the list in advance.)

Hint: Consider using two iterators, one fast and one slow.

Fred Note: The first time you solve this problem ignore the hint and use as much extra memory as you want. Then try to solve it in place with O(1) memory and use the hint.

In [ ]:
def find_cycle_naive(ll):
    prev_nodes = set()
    ll_cur = ll
    while ll_cur is not None:
        if ll_cur not in prev_nodes:
            prev_nodes.add(ll_cur)
            ll_cur = ll_cur.next
        else:
            return ll_cur
    return None

def find_loop_node(ll):
    ll_cur_slow = ll
    ll_cur_fast = ll.next
    while None not in [ll_cur_slow, ll_cur_fast]:
        if ll_cur_slow == ll_cur_fast:
            return ll_cur_slow
        ll_cur_slow = ll_cur_slow.next
        ll_cur_fast = ll_cur_fast.next
        ll_cur_fast = None if ll_cur_fast is None else ll_cur_fast.next
    return None

def find_loop_len(loop_node):
    len = 1
    cur_node = loop_node.next
    while cur_node != loop_node:
        len += 1
        cur_node = cur_node.next
    return len

def find_cycle_inplace(ll):
    loop_node = find_loop_node(ll)
    if loop_node is None:
        return None
    
    loop_len = find_loop_len(loop_node)
    print(loop_len)
    
    ll1_cur = ll
    ll2_cur = ll
    for i in range(loop_len):
        ll2_cur = ll2_cur.next
    while ll1_cur != ll2_cur:
        ll1_cur = ll1_cur.next
        ll2_cur = ll2_cur.next
    
    return ll1_cur

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 41, 4, 15, 5, 7])
lili1.next.next.next.next.next = lili1.next.next

# ll_cycle = find_cycle_naive(lili1)
ll_cycle = find_cycle_inplace(lili1)
print(ll_cycle.value)

## Delete a node from a singly linked list

Given a node in a singly linked list, deleting it in O(1) time appears impossible because its predecessor's next field has to be updated. Surprisingly, it can be done with one small caveat-the node to delete cannot be the last one in the list and it is easy to copy the value part of a node.

Write a program which deletes a node in a singly linked list. The input node is guaranteed not to be the tail node.

Hint: Instead of deleting the node, can you delete its successor and still achieve the desired configuration?

In [ ]:
def delete_node(ll):
    ll.value = ll.next.value
    ll.next = ll.next.next

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 41, 4, 15, 5, 7])
delete_node(lili1.next.next)
print(lili1)

## 7.8 Remove Duplicates from a Sorted List 

This problem is concerned with removing duplicates from a sorted list of integers. See Figure 7.8 for an example.

Write a program that takes as input a singly linked list of integers in sorted order, and removes duplicates from it. The list should be sorted.

Hint: Focus on the successor fields which have to be updated.

In [ ]:
def remove_duplicates(ll):
    ll_cur = ll
    while ll_cur is not None and ll_cur.next is not None:
        if ll_cur.value == ll_cur.next.value:
            ll_cur.next = ll_cur.next.next
        else:
            ll_cur = ll_cur.next
    return ll

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 4, 4, 5, 5, 5, 5, 5, 7])
remove_duplicates(lili1)
print(lili1)

## 8.1 Implement a Stack with Max API 

Design a stack that includes a max operation, in addition to push and pop. The max method should return the maximum value stored in the stack.

Hint: Use additional storage to track the maximum value.

In [ ]:
class Node:
    
    def __init__(self, val, next):
        self.val = val
        self.next = next

        
class Stack:
    
    def __init__(self):
        self._tail = None
        
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def to_list(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(node.val)
            node = node.next
        return vals[::-1]
        
    def push(self, val):
        self._tail = Node(val, self._tail)
        return self
    
    def pop(self):
        if self._tail is None:
            return None
        tail_val = self._tail.val
        self._tail = self._tail.next    
        return tail_val
    
    def peek(self):
        if self._tail is None:
            return None
        else:
            return self._tail.val
        
        
class MaxStack1:
    """
    time: O(1) push, O(N) pop
    space: O(N) overall, O(1) extra for max
    """
    
    def __init__(self):
        self._tail = None
        self._max = None
        
    def __str__(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(str(node.val))
            node = node.next
        vals = vals[::-1]
        return '->'.join(vals)
        
    def push(self, val):
        self._tail = Node(val, self._tail)
        self._max = max(self._max, val)
        return self
    
    def pop(self):
        tail_val = self._tail.val
        self._tail = self._tail.next
        
        if self._tail is None:
            self._max = None
        elif tail_val == self._max:
            self._max = self._tail.val
            node = self._tail.next
            while node is not None:
                self._max = max(node.val, self._max)
                node = node.next
        
        return tail_val
        
    def max(self):
        return self._max
    

class MaxStack2:
    """
    time: O(1) push, O(1) pop
    space: O(N) overall, O(N) extra for max
    """
    
    def __init__(self):
        self._stack = Stack()
        self._max_stack = Stack()
        
    def __str__(self):
        return str(self._stack)
        
    def push(self, val):
        self._stack.push(val)
        if self._stack.peek() >= self._max_stack.peek():
            self._max_stack.push(val)        
        return self
    
    def pop(self):
        tail_val = self._stack.pop()
        if tail_val == self._max_stack.peek():
            self._max_stack.pop()
        return tail_val
        
    def max(self):
        return self._max_stack.peek()
    
    
class MaxStack3:
    """
    time: O(1) push, O(1) pop
    space: O(N) overall, O(1) extra for max
    """
    
    def __init__(self):
        self._tail = None
        self._max = None
        
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def to_list(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(str(node.val))
            node = node.next
        return vals[::-1]
        
    def push(self, val):
        if self._max is None:
            self._max = val
        elif val > self._max:
            temp = 2 * val - self._max  # encode previous max in stored value
            self._max = val
            val = temp
        self._tail = Node(val, self._tail)
        return self
    
    def pop(self):
        tail_val = self._tail.val
        self._tail = self._tail.next
        if tail_val > self._max:
            prev_max = 2 * self._max - tail_val  # decode previous max      
            tail_val = self._max
            self._max = prev_max
        return tail_val
    
    def peek(self):
        if self._tail is None:
            return None
        else:
            return min(self._max, self._tail.val)
        
    def max(self):
        return self._max

In [ ]:
# s = MaxStack1()
# s = MaxStack2()
s = MaxStack3()

s.push(5)
print('max', s.max())
s.push(7)
print('max', s.max())
s.push(3)
print('max', s.max())
s.push(7)
print('max', s.max())

print(s)

print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())

## 8.3 Test a string over "{,},(,),[,]" for Well-Formedness 

A string over the characters "{,},(,),[,]" is said to be well-formed if the different types of brackets match in the correct order.

Write a program that tests if a string made up of these characters if well-formed.

Hint: Which left parenthesis does a right parenthesis match with?

In [ ]:
PAREN_MAP = {'}': '{', ')': '(', ']': '['}

def check_parens(s):
    paren_stack = Stack()
    for c in s:
        if c in PAREN_MAP.values():
            paren_stack.push(c)
        elif c in PAREN_MAP.keys():
            opening_paren = paren_stack.pop()
            if PAREN_MAP[c] != opening_paren:
                return False
    return paren_stack.peek() is None

In [ ]:
test_str = '{ab[()]c}'
check_parens(test_str)

## 8.6 Compute Buildings with a Sunset View 

You are given a series of buildings that have windows facing west. The buildings are in a straight line, and any building which is to the east of a building of equal or greater height cannot view the sunset.

Design an algorithm that processes buildings in east-to-west order and returns the set of buildings which view the sunset. Each building is specified by its height.

Hint: When does a building not have a sunset view?

In [ ]:
def get_sunset_view_buildings(heights):
    view_buildings = Stack()
    for height in heights:
        while height >= view_buildings.peek() and view_buildings.peek() is not None:
            view_buildings.pop()
        view_buildings.push(height)
    return view_buildings

east_to_west_heights = [5, 7, 4, 3, 3, 2]
print(get_sunset_view_buildings(east_to_west_heights).to_list())

## 8.8 Implement a Circular Queue 

A queue can be implemented using an array and two additional fields, the beginning and the end indices. This structure is sometimes referred to as a circular queue. Both enqueue and dequeue have O(1) time complexity. If the array is fixed, there is a maximum number of entries that can be stored. If the array is dynamically resized, the total time for m combined enqueue and dequeue operations is O(m).

Implement a queue API using an array for storing elements. Your API should include a constructor function, which takes as argument the initial capacity of the queue, enqueue and dequeue functions, and a function which returns the number of elements stored. Implement dynamic resizing to support storing an arbitrarily large number of elements.

Hint: Track the head and tail. How can you differentiate a full queue from an empty one?

In [ ]:
class CircularQueue1:
    
    def __init__(self, initial_capacity):
        self.initial_capacity = initial_capacity
        self._arr = [None] * initial_capacity
        self._head_ind = 0  # most recent
        self._tail_ind = 0  # least recent
        self._num_elements = 0
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def _advance_ind(self, ind):
        return ind + 1 if ind < len(self._arr) - 1 else 0
            
    def _resize_if_necessary(self):            
        if self._num_elements == len(self._arr):
            self._arr.append(None)
            if self._tail_ind > 0:
                for i in range(0, self._head_ind+1):
                    self._arr[i - 1], self._arr[i] = self._arr[i], None
                self._head_ind -= 1
                if self._head_ind < 0:
                    self._head_ind = len(self._arr) - 1
                
    def to_list(self):
        ind = self._tail_ind
        vals = []
        while True:
            vals.append(self._arr[ind])
            if ind == self._head_ind:
                break
            ind = self._advance_ind(ind)
        return vals[::-1]
        
    def enqueue(self, val):
        self._resize_if_necessary()
        if self._num_elements > 0:
            self._head_ind = self._advance_ind(self._head_ind)
        self._arr[self._head_ind] = val
        self._num_elements += 1
        return self
    
    def dequeue(self):
        val = self._arr[self._tail_ind]
        self._tail_ind = self._advance_ind(self._tail_ind)
        self._num_elements -= 1
        return val
    
    def get_num_elements(self):
        return self._num_elements
    
    
class CircularQueue2:
    
    def __init__(self, initial_capacity):
        self.initial_capacity = initial_capacity
        self._arr = [None] * initial_capacity
        self._head_ind = 0  # most recent
        self._tail_ind = 0  # least recent
        self._num_elements = 0
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def _advance_ind(self, ind):
        return ind + 1 if ind < len(self._arr) - 1 else 0
            
    def _resize_if_necessary(self):            
        if self._num_elements == len(self._arr):
            self._arr.extend([None] * self._num_elements)
            for i in range(self._tail_ind):
                self._arr[self._num_elements + i] = self._arr[i]
            self._head_ind = self._tail_ind + self._num_elements - 1
                
    def to_list(self):
        ind = self._tail_ind
        vals = []
        while True:
            vals.append(self._arr[ind])
            if ind == self._head_ind:
                break
            ind = self._advance_ind(ind)
        return vals[::-1]
        
    def enqueue(self, val):
        self._resize_if_necessary()
        if self._num_elements > 0:
            self._head_ind = self._advance_ind(self._head_ind)
        self._arr[self._head_ind] = val
        self._num_elements += 1
        return self
    
    def dequeue(self):
        val = self._arr[self._tail_ind]
        self._tail_ind = self._advance_ind(self._tail_ind)
        self._num_elements -= 1
        return val
    
    def get_num_elements(self):
        return self._num_elements

In [ ]:
q = CircularQueue2(5)
print(q._arr)
q.enqueue(5)
print(q._arr)
q.enqueue(8)
print(q._arr)
q.enqueue(4)
q.enqueue(4)
q.enqueue(4)
q.enqueue(4)
print(q._arr)
q.dequeue()
print(q._arr)
q.enqueue(10)
print(q._arr)
q.enqueue(11)
print(q._arr)
print(q)

## 8.10 Implement a Queue with Max API 

Implement a queue with enqueue, dequeue, and max operations. The max operation returns the maximum element currently in the queue.

Hint: When can an element never be returned by max, regardless of future updates?

In [ ]:
class Node:
    
    def __init__(self, val, next, prev=None):
        self.val = val
        self.next = next
        self.prev = prev

        
class Queue:
    
    def __init__(self):
        self._back = None
        self._front = None
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
        
    def to_list(self):
        vals = []
        node = self._back
        while node is not None:
            vals.append(node.val)
            node = node.next
        return vals
        
    def enqueue(self, val):
        self._back = Node(val, self._back)
        if self._front is None:
            self._front = self._back
        else:
            self._back.next.prev = self._back
        return self
    
    def dequeue(self):
        val = self._front.val
        self._front = self._front.prev
        if self._front is None:
            self._back = None
        else:
            self._front.next = None
        return val


class MaxQueue1:
    
    def __init__(self):
        self._queue = Queue()
        self._max = None
    
    def __str__(self):
        return str(self._queue)
        
    def to_list(self):
        return self._queue.to_list()
        
    def enqueue(self, val):
        self._max = max(self._max, val)
        self._queue.enqueue(val)
        return self
    
    def dequeue(self):
        val = self._queue.dequeue()
        if val == self._max:
            self._max = max(self.to_list())
        return val
    
    def max(self):
        return self._max
    
    
class MaxQueue2:
    
    def __init__(self):
        self._stack1 = MaxStack3()
        self._stack2 = MaxStack3()
    
    def __str__(self):
        return str('->'.join([str(val) for val in self.to_list()]))
        
    def to_list(self):
        return self._stack1.to_list() + self._stack2.to_list()
        
    def enqueue(self, val):
        self._stack1.push(val)
        return self
    
    def dequeue(self):
        if self._stack2.peek() is None:
            while self._stack1.peek() is not None:
                self._stack2.push(self._stack1.pop())
        return self._stack2.pop()
    
    def max(self):
        return max(self._stack1.max(), self._stack2.max())

In [ ]:
q = MaxQueue2()
q.enqueue(5)
q.enqueue(8)
q.enqueue(7)
q.enqueue(6)
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())

## 9.1 Check if a binary tree is height-balanced

A binary tree is said to be height-balanced if for each node in the tree, the difference in height of its left and right subtrees is at most one. A perfect binary tree is height-balanced, as is a complete binary tree. A height-balanced binary tree does not have to be perfect or complete -- see Figure 9.2 on the following page for an example.

Write a program that takes as input the root of a binary tree and checks whether the tree is height-balanced.

Hint: Think of a classic binary tree algorithm.

In [ ]:
class BinaryTree:
    
    def __init__(self, key=None, val=None):
        self.key = key
        self.val = val
        self.parent = None
        self.left = None
        self.right = None
        
    def insert(self, key, val=None):
        if self.key is None:
            self.key = key
            self.val = val
            return self
        
        if key < self.key:
            if self.left is None:
                self.left = BinaryTree(key, val)
                self.left.parent = self
            else:
                self.left.insert(key, val)
        else:
            if self.right is None:
                self.right = BinaryTree(key, val)
                self.right.parent = self
            else:
                self.right.insert(key, val)
        
        return self
    
    def get_max_depth(self, prev_depth=0):
        if self.left is None:
            left_depth = prev_depth + 1
        else:
            left_depth = self.left.get_max_depth(prev_depth+1)
        
        if self.right is None:
            right_depth = prev_depth + 1
        else:
            right_depth = self.right.get_max_depth(prev_depth+1)
            
        depth = max(left_depth, right_depth)
        
        return depth
    
    def get_min_depth(self, prev_depth=0):
        if self.left is None or self.right is None:
            return prev_depth + 1
        else:
            return min(self.left.get_min_depth(prev_depth+1), self.right.get_min_depth(prev_depth+1))

        
def is_height_balanced(binary_tree):
    max_diff = binary_tree.get_max_depth() - binary_tree.get_min_depth()
    return max_diff <= 1

In [ ]:
b = BinaryTree().insert(5).insert(6).insert(3)

In [ ]:
is_height_balanced(b)

## 9.2 Test if a binary tree is symmetric

A binary tree is symmetric if you can draws a vertical line through the root and then the left subtree is the mirror image of the right subtree. The concept of a symmetric binary tree is illustrated in Figure 9.3.

Write a program that checks whether a binary tree is symmetric.

Hint: The definition of symmetry is recursive.

In [ ]:
def is_symmetric(binary_tree):
    current_nodes = [binary_tree]
    
    while None not in current_nodes:
        for i in range(len(current_nodes) // 2):
            if current_nodes[i].key != current_nodes[len(current_nodes) - 1 - i].key:
                return False
        
        child_nodes = []
        for node in current_nodes:
            child_nodes.extend([node.left, node.right])
        
        current_nodes = child_nodes
    
    return True

In [ ]:
b1 = BinaryTree().insert(5).insert(6).insert(3)
b2 = BinaryTree(5)
b2.left = BinaryTree(8)
b2.right = BinaryTree(8)
is_symmetric(b2)

## 9.4 Compute the LCA when nodes have parent pointers

Given two nodes in a binary tree, design an algorithm that computes their lowest common ancestor. Assume that each node has a parent pointer.

Hint: The problem is easy if both nodes are the same distance from the root.

## 9.11 Implement an inorder traversal with O(1) space

The direct implementation of an inorder traversal using recursion has O(h) space complexity, where h is the height of the three. Recursion can be removed with an explicit stack, but the space complexity remains O(h).

Write a nonrecursive program for computing the inorder traversal sequence for a binary tree. Assume nodes have parent fields.

Hint: How can you tell whether a node is a left child or right child of its parent?

## 9.12 Reconstruct a binary tree from traversal data

Many different binary trees yield the same sequence of keys in an inorder, preorder or postorder traversal. However, given an inorder traversal and one of any two other traversal orders of a binary tree, there exists a unique binary tree that yields those orders, assuming each node holds a distinct key. For example, the unique binary tree whose inorder traversal sequence is <F B A E H C D I G> and whose preorder traversal sequence is <H B F E A C D G I> is given in Figure 9.5 on the facing page.

Given an inorder traversal sequence and a preorder traversal sequence of a binary tree write a program to reconstruct the tree. Assume each node has a unique key.

Hint: Focus on the root.

## 9.16 Compute the right sibling tree

For this problem, assume that each binary tree node has an extra field, call it level-next, that holds a binary tree node (this field is distinct from the fields for the left and right children). The level-next field will be used to compute a map from nodes to their right siblings. The input is assumed to be a perfect binary tree. See Figure 9.6 for an example.

Write a program that takes a perfect binary tree, and sets each node's level-next field to the node on its right, if one exists.

Hint: Think of an appropriate traversal order.

## 10.1 Merge Sorted Files

Write a program that takes as input a set of sorted sequences and computes the union of these sequences as a sorted sequence. For example, if the input is <3, 5, 7>, <0, 6>, and <0, 6, 28>, then the output is <0, 0, 3, 5, 6, 6, 7, 28>

In [ ]:
class BinaryHeap:
    
    def __init__(self, min=True):
        self.arr = []
        self.size = 0
        self._sign = 2*min - 1
        
    def _get_val(self, ind):
        return self.arr[ind][0]
        
    def push(self, val, data=None):
        val *= self._sign
        if self.size == len(self.arr):
            self.arr.append((val, data))
        else:
            self.arr[self.size] = (val, data)
        self.size += 1
        
        cur_ind = self.size - 1
        while cur_ind > 0:
            par_ind = cur_ind // 2
            if self._get_val(par_ind) > self._get_val(cur_ind):
                self.arr[par_ind], self.arr[cur_ind] = self.arr[cur_ind], self.arr[par_ind]
                cur_ind = par_ind
            else:
                break
                
        return self
    
    def pop(self):
        min_entry = (self.arr[0][0] * self._sign, self.arr[0][1])
        self.arr[0] = self.arr[self.size - 1]
        self.arr[self.size - 1] = '-'
        self.size -= 1
        
        cur_ind = 0
        while cur_ind <= self.size // 2:
            # Find index of smallest child
            child1_ind = (cur_ind + 1) * 2 - 1
            child2_ind = (cur_ind + 1) * 2
            if child1_ind < self.size:
                min_ind = child1_ind
            else:
                break
            if child2_ind < self.size:
                min_ind = min_ind if self._get_val(min_ind) < self._get_val(child2_ind) else child2_ind
            
            # Swap
            if self._get_val(cur_ind) > self._get_val(min_ind):
                self.arr[min_ind], self.arr[cur_ind] = self.arr[cur_ind], self.arr[min_ind]
                cur_ind = min_ind
            else:
                break

        return min_entry
    
    
    
def get_union(sorted_lists):
    """
    time: O{Nlog(M)}, where N is length of combined output list and M is number of input lists
    space: O{N}
    """
    heap = BinaryHeap()
    final_list = []
    
    for sorted_list in sorted_lists:
        heap.push(sorted_list[0], sorted_list[1:])
        
    while heap.size > 0:
        min_val, remaining_vals = heap.pop()
        final_list.append(min_val)
        if len(remaining_vals) > 0:
            heap.push(remaining_vals[0], remaining_vals[1:])
            
    return final_list

In [ ]:
lists = [[3, 5, 7], [0, 6], [0, 6, 28]]
get_union(lists)

In [ ]:
x = BinaryHeap(min=False)
x.push(5)
x.push(3)
x.push(1)
x.push(2)
x.pop()
x.arr

In [ ]:
sorted_vals = []
while x.size > 0:
    sorted_vals.append(x.pop()[0])
sorted_vals

## 10.3 Sort an Almost-Sorted Array

Often data is almost-sorted -- for example, a server receives timestamped stock quotes and earlier quotes may arrive slightly after later quotes because of differences in server loads and network routes. In this problem we address efficient ways to sort such data.

Write a program which takes as input a very long sequence of numbers and prints the numbers in sorted order. Each number is at most k away from its correctly sorted position. *such an array is sometimes referred to as being k-sorted) For example, no number in the sequence (3,-1,2,6,4,5,8) Is more than 2 away from its final sorted position.

Hint: How many numbers must you read after reading the ith number to be sure you can place it in the correct location?

In [ ]:
def sort_almost_sorted(vals, k):
    heap = BinaryHeap(min=True)
    sorted_vals = []
    
    for val in vals:
        heap.push(val)
        if heap.size > 2*k + 1:
            sorted_vals.append(heap.pop()[0])
            
    while heap.size > 0:
        sorted_vals.append(heap.pop()[0])
    
    return sorted_vals

In [ ]:
vals = (3,-1,2,6,4,5,8)
sort_almost_sorted(vals, 2)

## 10.4 Compute the k closest stars

Consider a coordinate system for the Milky Way, in whcih earth is at (0,0,0). Model stars as points, and assume distances are in light years. The Milky Way consists of approximately 10^12 stars, and their coordinates are stored in a file.

How would you compute the k stars which are closest to Earth?

Hint: Suppose you know the k closest stars in the first n stars. If the (n+1)st star is to be added to the set of k closes stars, which element in the set should be evicted?

In [ ]:
import math

def find_closest_stars(star_coords, k):
    heap = BinaryHeap(min=False)
    for star_coord in star_coords:
        star_dist = math.sqrt(star_coord[0]**2 + star_coord[1]**2 + star_coord[2]**2)
        heap.push(star_dist, star_coord)
        if heap.size > k:
            heap.pop()
            
    closest_coords = []
    while heap.size > 0:
        closest_coords.append(heap.pop()[1])
    
    return closest_coords[::-1]

In [ ]:
dists = [(4,46,7), (3,6,6), (8,43,66), (4,6,57), (8,7,5)]
find_closest_stars(dists, 3)

## 11.1 Search a sorted array for the first of k

Binary search commonly asks for the index of any element of a sorted array that is equal to a specified element. The following problem has a slight twist on this.

Write a method that takes a sorted array and a key and returns the index of the first occurrence of that key in the array. For example, when applied to the array in Figure 11.1 your algorithm should return 3 if the given key is 108; if it is 285, your algorithm should return 6.

Hint: What happens when every entry equals k? Don't stop when you first see k.

In [47]:
def find_first_occurence_ind(arr, key):
    ceil = len(arr)
    floor = 0
    keep_searching = True

    while keep_searching:
        test_ind = (floor + ceil) // 2
        test_val = arr[test_ind]
        if test_ind in (floor, ceil):
            keep_searching = False

        if test_val >= key:
            ceil = test_ind
        else:
            floor = test_ind

    return ceil

In [48]:
arr = [3,4,4,6,6,6,6,7,8]
key = 6
find_first_occurence_ind(arr, key)

3

## 11.4 Compute the integer square root

Write a program which takes a nonnegative integer and returns the largest integer whose square is less than or equal to the given integer. For example, if the input is 16, return 4; if the input is 300, return 17, since 17^2 = 289 < 300 and 18^2 = 324 > 300.

Hint: look out for a corner case!

In [49]:
def compute_int_sqrt(num):
    floor = 0
    ceil = num
    int_sqrt = 0
    keep_searching = True

    while keep_searching:
        test_val = (ceil + floor + 1) // 2
        if test_val in (floor, ceil):
            keep_searching = False

        if test_val**2 > num:
            ceil = test_val
        else:
            floor = test_val
            int_sqrt = test_val

    return int_sqrt

In [50]:
compute_int_sqrt(0)

0

## 11.8 Find the kth largest element

Many algorithms require as a subroutine the computation of the kth largest element of an array. The first largest element is simply the largest element. The nth largest element is the smallest element, where n is the length of the array.

For example, if the input array A = <3,2,1,5,4> then A[3] is the first largest element in !, A[0] is the third largest element in A, and A[2] is the fifth largest element in A.

Design an algorithm for computing the kth largest element in an array. Assume entries are distinct.

Hint: Use divide and conquer in conjunction with randomization

## 12.2 Is an anonymous letter constructible?

Write a program which takes text from an anonymous letter and text from a magazine and determines if it is possible to write the anonymous letter using the magazine. The anonymous letter can be written using the magazine if for each character in the anonymous letter, the number of times it appears in the anonymous letter is no more than the number of times it appears in the magazine.

Hint: Count the number of distinct characters appearing in the letter.

In [43]:
from collections import defaultdict

def is_constructible(letter, magazine):
    letter_counter = defaultdict(int)
    remaining_chars = len(letter)
    
    for letter_char in letter:
        letter_counter[letter_char] += 1
        
    for mag_char in magazine:
        if letter_counter.get(mag_char, 0) > 0:
            letter_counter[mag_char] -= 1
            remaining_chars -= 1
            if remaining_chars == 0:
                break
            
    return remaining_chars == 0

## 12.6 Find the Nearest repeated entries in an array

People do not like reading text in which a word is used multiple times in a short paragram. You are to write a program which helps identify such a problem.

Write a program which takes as input an array and finds the distance between a closest pair of equal entries. For example, if s = " all work and no play makes for no work no fun and no results" then the second and third occurrences of "no" is the closest pair.

Hint: Each entry in the array is a candidate.

In [55]:
def find_nearest(s):
    s = s.split()
    d = {}
    min_sep = None
    min_tok = None

    for i, tok in enumerate(s):
        if tok in d:
            sep = i - d[tok]
            if sep < min_sep or min_sep is None:
                min_sep = sep
                min_tok = tok
        d[tok] = i

    return min_tok, min_sep

In [56]:
s = "all work and no play makes for no work no fun and no results"
find_nearest(s)

('no', 2)

## 12.3 Implement an ISBN Cache

The International Standard Book Number (ISBN) is a unique commercial book identifier. It is a string of length 10. The first 9 characters are digits; the last character is a check character. The check character is the sum of the first 9 digits, mod 11, where with 10 represented by 'X' (Modern ISBNs use 13 digits, and the check digit is taken mod 10; this problem is concerned with 10-digit ISBNs)

Create a cache for looking up prices of books identified by their ISBN. You implement lookup, insert, and remove methods. Use the Least Recently Used (LRU) policy for cache eviction. If an ISBN is already present, insert should not change the price, but it should update that entry to be the most recently used entry. Lookup should also update that entry to be the most recently used entry.

Hint: Amortize the cost of deletion. Alternatively, use an auxiliary data structure.

In [51]:
from collections import OrderedDict

class LRUCache(object):
    
    def __init__(self, size_limit):
        self.size_limit = size_limit
        self._cache = OrderedDict()
        
    def _update(self, key):
        if key in self._cache:
            val = self._cache[key]
            self.remove(key)
            self._cache[key] = val
        return self
        
    def insert(self, key, val):
        if key in self._cache:
            self._update(key)
        else:
            if len(self._cache) == self.size_limit:
                self._cache.popitem(0)
            self._cache[key] = val
        return self
    
    def lookup(self, key):
        self._update(key)
        return self._cache.get(key)
    
    def remove(self, key):
        self._cache.pop(key)
        return self

In [ ]:
x = LRUCache(3)
x.insert('a', 1).insert('b', 2).insert('c', 3).insert('d', 4).insert('b', 2)
x._cache

## 13.1 Compute The Intersection of Two Sorted Arrays

A natural implementation for a search engine is to retrieve documents that match the set of words in a query by maintaining an inverted index. Each page is assigned an integer identifier, its documentID. An inverted index is a mapping that takes a word w and returns a sorted array of page-ids which contain w-- the sort order could be, for example, the page rank in descending order. When a query contains multiple words, the search engine finds the sorted array for each word and then computes the intersection of these arrays-- these are the pages containing all the words in the query. The most computationally intensive step of doing this is finding the intersection of the sorted arrays.

Write a program which takes as input two sorted arrays, and returns a new array containing elements that are present in both of the input arrays. The input arrays may have duplicate entries, but the returned array should be free of duplicates. For example, the input is <2,3,3,4,4,5,6,6,8,12> and <5,5,6,8,8,9,10,10> your output should be <5,6,8>

Hint: Solve the problem if the input array lenghts differ by orders of magnitude. What if they are approximately equal?

In [57]:
def find_intersection(arr1, arr2):
    ind1 = 0
    ind2 = 0
    intersect_vals = []

    while ind1 < len(arr1) and ind2 < len(arr2):
        if arr1[ind1] == arr2[ind2]:
            if len(intersect_vals) == 0 or arr1[ind1] > intersect_vals[-1]:
                intersect_vals.append(arr1[ind1])
            ind1 += 1
            ind2 += 1
        elif arr1[ind1] > arr2[ind2]:
            ind2 += 1
        else:
            ind1 += 1

    return intersect_vals

In [58]:
arr1 = [2,3,3,4,4,5,6,6,8,12]
arr2 = [5,5,6,8,8,9,10,10]
find_intersection(arr1, arr2)

[5, 6, 8]

## 13.2 Merge Two Sorted Arrays

Suppose you are given two sorted arrays of integers. If one array has enough empty entries at its end, it can be used to store the combined entries of the two arrays in sorted order. For example, consider < 5, 13, 17, _ _ _ _ _ > and <3,7,11,19> where _ denotes an empty entry. Then the combined sorted entries can be stored in the first array as <3,5,7,11,13,17,19, _>

Write a program which takes as input two sorted arrays of integers, and updates the first to the combined entries of the two arrays in sorted order. Assume the first array has enough empty entries at its end to hold the results.

Hint: Avoid repeatedly moving entries

In [59]:
def merge_arrays(arr1, arr2):
    # Get size of arrays
    num1 = 0
    for val in arr1:
        if val == '_':
            break
        num1 += 1
    num2 = 0
    for val in arr2:
        if val == '_':
            break
        num2 += 1
    
    # Merge starting from the back
    while num1 > 0 and num2 > 0:
        val1 = arr1[num1 - 1]
        val2 = arr2[num2 - 1]

        if val1 > val2:
            arr1[num1 + num2 - 1] = val1
            num1 -= 1
        else:
            arr1[num1 + num2 - 1] = val2
            num2 -= 1
    
    # Add the remainder of the second array if necessary
    if num2 > 0:
        arr1[:num2] = arr2[:num2]

In [60]:
arr1 = [8, 13, 17, '_', '_', '_', '_', '_']
arr2 = [3, 7, 11, 19]
merge_arrays(arr1, arr2)
arr1

[3, 7, 8, 11, 13, 17, 19, '_']

## 14.1 Test If a Binary Tree Satisfies the BST Property

Write a program that takes as input a binary tree and checks if the tree satisfiest the BST property.

Hint: Is it correct to check for each node that its key is greater than or equal to the key at its left child and less than or equal to the key at its right child?

## 14.2 Find the First Key Greater than a Given Value in a BST

Write a program that takes as input a BST and a value, and returns the first key that would appear in an inorder traversal which is greater than the input value. For example, when applied to the BST in Figure 14.1 on Page 202 you should return 29 for input 23. 

## 14.3 Find the k largest Elements in a BST 

A BST is a sorted data structure, which suggests that it should be possible to find the k largest keys easily.

Write a program that takes as input a BST and an integer k, and returns the k largest elements in the BST in decreasing order. For example, if the input is the BST in figure 14.1 on Page 202 and k = 3, your program should return <53,47,43>.

Hint: What does an inorder traversal yield?

## 15.1 The Towers of Hanoi Problem

A peg contains rings in sorted order, with the largest right being the lowest. You are to transfer these rings to another peg, which is initially empty. This is illustrated in Figure 15.2

Write a program which prints a sequence of operations that transfer n rings from one peg to another. You have a third peg, which is initially empty. The only operation you can perform is taking a single ring from the top of one peg and placing it on the top of another peg. You must never place a larger ring above a smaller ring.

Hint: If you know how to transfer the top n-1 rings, how does that help move the nth ring?

In [81]:
import copy

def transfer_rings(num_rings):
    peg1 = range(num_rings, 0, -1)
    peg2 = []
    peg3 = []
    peg_history = [(copy.copy(peg1), copy.copy(peg2), copy.copy(peg3))]
    
    while len(peg1) > 0:
        peg2.append(peg1.pop())
        peg_history.append((copy.copy(peg1), copy.copy(peg2), copy.copy(peg3)))
        peg2.extend(peg3)
        peg3 = []
        peg_history.append((copy.copy(peg1), copy.copy(peg2), copy.copy(peg3)))
        if len(peg1) > 0:
            peg3 = peg2
            peg2 = []
            peg_history.append((copy.copy(peg1), copy.copy(peg2), copy.copy(peg3)))
    
    return peg_history

In [83]:
peg_history = transfer_rings(3) 
for hist in peg_history:
    print(hist)

([3, 2, 1], [], [])
([3, 2], [1], [])
([3, 2], [1], [])
([3, 2], [], [1])
([3], [2], [1])
([3], [2, 1], [])
([3], [], [2, 1])
([], [3], [2, 1])
([], [3, 2, 1], [])


## 15.2 Generate All Nonattacking Placements of n-Queens 

A nonattacking placement of queens is one in which no two queens are in the same row, column, or diagonal. See Figure 15.4 for an example.

Write a program which returns all distinct nonattacking placements of n queens on an n x n chessboard, where n is an input to the program.

Hint: If the first queen is placed at (i,j) where can the remaining queens definitely not be placed?

In [137]:
from copy import copy

def on_diag(p1, p2):
    try:
        return abs(float(p1[0] - p2[0]) / float(p1[1] - p2[1])) == 1
    except ZeroDivisionError:
        return False

def not_on_any_diag(p, p_prev):
    for p2 in p_prev:
        if on_diag(p, p2):
            return False
    return True

def get_valid_points(row_inds, prev_seqs):
    all_valid_seqs = copy(prev_seqs)
    col_ind = len(row_inds) - 1
    
    for i in range(len(row_inds)):
        row_ind = row_inds[i]
        test_point = (row_ind, col_ind)
                
        new_seqs = []
        for seq in prev_seqs:
            if not_on_any_diag(test_point, seq):
                new_seqs.append(seq + [test_point])
                        
        row_inds_new = copy(row_inds)
        row_inds_new.pop(i)
        
        valid_seqs = get_valid_points(row_inds_new, new_seqs)        
        all_valid_seqs.extend(valid_seqs)
        
    return all_valid_seqs

def generate_nonattacking_placements(n):
    seqs = get_valid_points(range(n), [[]])
    return [seq for seq in seqs if len(seq) == n]

In [139]:
n = 4
generate_nonattacking_placements(n)

[[(1, 3), (3, 2), (0, 1), (2, 0)], [(2, 3), (0, 2), (3, 1), (1, 0)]]

## 16.1 Count the Number of Score Combinations 

In an American football game, a play can lead to 2 points (safety), 3 points (field goal) or 7 points (touchdown, assuming the extra point). Many different combinations of 2,3 and 7 point plays can make up a final score. For example, four combinations of plays yield a score of 12.

* 6 safeties (2 x 6 = 12)
* 3 safeties and 2 field goals (2 x 3 + 3 x 2 = 12)
* 1 safety, 1 field goal and 1 touchdown (2 x 1 + 3 x 1 + 7 x 1 = 12), and 
* 4 field goals (3 x 4 = 12)

Write a program that takes a final score for individual plays, and returns the number of combinations of plays that result in the final score.

Hint: Count the number of combinations in which there are 0 w_0 plays then 1 w_) plays, etc.

In [150]:
# num_ways[n] = num_ways[n-2] + num_ways[n-3] + num_ways[n-7]

n = 12

num_ways_counter = [0] * (n + 1)
num_ways_counter[2] = 1
num_ways_counter[3] = 1
num_ways_counter[7] = 1

for i in range(6, n+1):
    num2 = 0 if i-2 < 0 else num_ways_counter[i-2]
    num3 = 0 if i-3 < 0 else num_ways_counter[i-3]
    num7 = 0 if i-7 < 0 else num_ways_counter[i-7]
    
    num_ways_counter[i] = num2 + num3 + num7

nn = n
num_ways_counter[:nn+1]

[0, 0, 1, 1, 1, 1, 2, 2, 3, 5, 6, 9, 12]

## 16.2 Compute the Levenshtein Distance 

Spell checkers make suggestions for misspelled words. Given a misspelled string, a spell checker should return words in the dictionary which are close to the misspelled string.

In 1965 Vladimir Levenshtien defined the distance between two words as the minimum number of "edits" it would take to transform the misspelled word into a correct word, where a single edit is the insertion, deletion or substitution of a single character. For example, the Levenshtien distance between "Saturday" and "sundays" is 4 - delete the first 'a' and 't', substitute 'r' by 'n'n and insert the trailing 's'.

Write a program that takes two strings and computes the minimum number of edits needed to transform the first string into the second string.

Hint: Consider the same problem for prefixes of the two strings.